In [84]:
import pip

def import_or_install(package):
    try:
        __import__(package)
    except ImportError:
        pip.main(['install', package])



packages = ['nltk', 'pyvis' , 'matplotlib_inline' ,'matplotlib', 'gensim.corpora','gensim.utils','gensim.models' , 'matplotlib_inline.pyplot' ,'pandas', 'numpy', 'umap','sqlite3','spacy','win32com.client','datetime','pyvis.network','matplotlib.pyplot','plotly.graph_objects','scipy','networkx','dash','gensim','logging','warnings','nltk.corpus']
for package in packages:
    import_or_install(package)


print('\n done with imports')




Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
ERROR: Could not find a version that satisfies the requirement matplotlib_inline.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib_inline.pyplot
You should consider upgrading via the 'c:\users\bensg\pycharmprojects\network_groupdetection\venv\scripts\python.exe -m pip install --upgrade pip' command.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


  Using cached spacy-3.2.3-cp39-cp39-win_amd64.whl (11.3 MB)
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Using cached typer-0.4.0-py3-none-any.whl (27 kB)
  Using cached numpy-1.22.3-cp39-cp39-win_amd64.whl (14.7 MB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached cymem-2.0.6-cp39-cp39-win_amd64.whl (36 kB)
  Using cached pathy-0.6.1-py3-none-any.whl (42 kB)
  Using cached tqdm-4.63.0-py2.py3-none-any.whl (76 kB)
  Using cached thinc-8.0.15-cp39-cp39-win_amd64.whl (1.0 MB)
  Using cached wasabi-0.9.0-py3-none-any.whl (25 kB)
  Using cached blis-0.7.6-cp39-cp39-win_amd64.whl (6.6 MB)
  Using cached srsly-2.4.2-cp39-cp39-win_amd64.whl (451 kB)
  Using cached catalogue-2.0.6-py3-none-any.whl (17 kB)
  Using cached pydantic-1.8.2-cp39-cp39-win_amd64.whl (1.9 MB)
  Using cached spacy_loggers-1.0.1-py3-none-any.whl (7.0 kB)
  Using cached preshed-3.0.6-cp39-cp39-win_amd64.whl (112 kB)
  Using cached murmurhash-1.0.6-cp39-cp39-win_amd64.whl (21 kB)
  

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\bensg\\pycharmprojects\\network_groupdetection\\venv\\Lib\\site-packages\\numpy\\.libs\\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll'
Check the permissions.

You should consider upgrading via the 'c:\users\bensg\pycharmprojects\network_groupdetection\venv\scripts\python.exe -m pip install --upgrade pip' command.



 done with imports


In [93]:
import nltk
import umap as umap
import sqlite3
nltk.download('stopwords')
#parser = English()

# import spacy
# from spacy.lang.en import English

import pandas as pd
from pandas import Series

import numpy as np
#import pywin32
import win32com.client
import networkx as nx
from datetime import datetime, timedelta
from pyvis.network import Network
import matplotlib_inline
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy import stats

import networkx as nx

import dash
from dash import dcc
from dash import html

#G = nx.random_geometric_graph(200, 0.125)
import scipy

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
# import spacy

% matplotlib inline

# Enable logging for gensim - optional
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
stopwords = nltk.corpus.stopwords.words('english')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bensg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
UsageError: Line magic function `%` not found.


In [94]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\bensg\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Run only for latest

In [120]:
#Get messages
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

inbox = outlook.GetDefaultFolder(6)
print(inbox.Name)

messages = inbox.Items

Inbox


In [96]:
#MessageObject filters

received_dt = datetime.now() - timedelta(days=100)
received_dt = received_dt.strftime('%m/%d/%Y %H:%M %p')
messages = messages.Restrict("[ReceivedTime] >= '" + received_dt + "'")

#messages = messages.Restrict("[SenderEmailAddress] = 'navjyot.kaur@ConnectYourCare.com'")

In [121]:
#Build core tables
pd_messages = []
pd_recipients = []

for message in list(messages):
    try:
        recipients_list = message.Recipients
        recipients_cleaned = ""
        for recipient in recipients_list:
            pd_recipients.append({
                "entryID": str(message.EntryID),
                "sender": str(message.Sender),
                "recipient": str(recipient),
            })

        pd_messages.append({
            "entryID": str(message.EntryID),
            "conversationID": str(message.ConversationID),
            "conversationIndex": str(message.ConversationIndex),
            "createTime": str(message.CreationTime),
            "recievedTime": str(message.ReceivedTime),
            "ConversationTopic": str(message.ConversationTopic),
            "subject": str(message.Subject),
            "body": str(message.body)
        })
    except:
        print("error =  " + str(recipients_list))



error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error =  <COMObject <unknown>>
error = 

In [122]:

df_messages = pd.DataFrame(pd_messages)
df_recipients = pd.DataFrame(pd_recipients)

print(df_messages.shape)


(2474, 8)


In [123]:
#Text normalization stage

#import textblob & define stopwords. Not sure why when I put it in the main import block it then fails to get detected in this block.
from textblob import TextBlob
stopwords = nltk.corpus.stopwords.words('english')

#Dont think I need this function anymore
def tag_corp(corp, tag):
    return gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(corp), ([tag]))


#Get each words Part of speech ( verb, noun etc) then pass it with the word to a lemmatizer. Then put the lemmatized word back into the Pandas DF row
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a',
                "N": 'n',
                "V": 'v',
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)


#convert to lowercase
df_messages['temp_body'] = df_messages['body'].apply(lambda x: x.lower())

#Get rid of punctuation (\r and \n and _ were still showing up, ill need to fix that at some point, for now they are just extra rows)
df_messages['temp_body'] = df_messages['temp_body'].str.replace(r'[^\w\s]+', '')  #(?:\w+)
df_messages['temp_body'] = df_messages['temp_body'].replace(r'\n', ' ', regex=True)
df_messages['temp_body'] = df_messages['temp_body'].replace(r'\r', ' ', regex=True)
df_messages['temp_body'] = df_messages['temp_body'].replace(r'_', '', regex=True)

#Strip away whitespaces
df_messages['temp_body'] = df_messages['temp_body'].str.strip()

#Remove stopwords
df_messages['temp_body'] = df_messages['temp_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

#Lemmatize remaining words
df_messages['temp_body'] = df_messages['temp_body'].apply(lemmatize_with_postag)

#bigram geneation
# df_messages['bigrams'] = df_messages['temp_body'].apply(lambda row: list(nltk.bigrams(row.split(' '))))

df_messages[['body','temp_body']]

# need to drop na's

C:\Users\bensg\AppData\Local\Temp\ipykernel_19152\3949661617.py:28: FutureWarning:

The default value of regex will change from True to False in a future version.



,body,temp_body
0,I think it looks good. Can you reach out to N...,think look good reach nancy directly approve b...
1,\t\r\n\t\r\n\tThank you for electronically sig...,thank electronically sign movement earth trek ...
2,Congratulations! Your email address has been c...,congratulation email address confirm copy move...
3,‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌...,httpstwittercomiredirecturlhttps3a2f2ftwitterc...
4,"fyi, the core group of old guys are all out to...",fyi core group old guy today jon andy hopefull...
...,...,...
2469,\t <https://www.google.com/alerts?source=alert...,httpswwwgooglecomalertssourcealertsmailhlenglu...
2470,\t <https://www.google.com/alerts?source=alert...,httpswwwgooglecomalertssourcealertsmailhlenglu...
2471,\t <https://www.google.com/alerts?source=alert...,httpswwwgooglecomalertssourcealertsmailhlenglu...
2472,<http://www.causes.com/assets/email/color_hea...,httpwwwcausescomassetsemailcolorheaderwwhite13...


In [124]:
#THIS CHUNK SAVES TO SQLITE THE DATA PULLED FROM OUTLOOK

sql_con = sqlite3.connect("test_db_sqlite.db")

# Write the new DataFrame to a new SQLite table
df_messages.to_sql("messages", sql_con, if_exists="replace")
# df_recipients = pd.DataFrame(pd_recipients)
df_recipients.to_sql("recipients", sql_con, if_exists="replace")

sql_con.close()

START HERE

In [125]:
#Start from here to cut down on processing time

sql_con = sqlite3.connect("test_db_sqlite.db")




message_query = 'select * from messages;'
recipient_query = 'select recipients.* from recipients;'
df_messages = pd.read_sql_query(message_query, sql_con)
df_recipients = pd.read_sql_query(recipient_query, sql_con)

sql_con.close()

df_recipients = df_recipients.drop('index', 1)
df_messages = df_messages.drop('index', 1)


C:\Users\bensg\AppData\Local\Temp\ipykernel_19152\2966655317.py:15: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

C:\Users\bensg\AppData\Local\Temp\ipykernel_19152\2966655317.py:16: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



In [126]:
#This chunk here creates the topics

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

tfidf = TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1',
                        stop_words='english')

dtm = tfidf.fit_transform(df_messages['temp_body'])
topic_count = 15

nmf_model = NMF(n_components=topic_count, random_state=42)
nmf_model.fit(dtm)
print("record size : ", len(tfidf.get_feature_names_out()))

topic_text_dict = {}
for index, topic in enumerate(nmf_model.components_):
    # print(f'The top 15 words for topic #{index}')
    topic_text_string =  str([tfidf.get_feature_names_out()[i] for i in topic.argsort()[-topic_count:]])
    v = {index: str(topic_text_string)}
    topic_text_dict.update(v)


topic_text_df =  pd.Series(topic_text_dict).to_frame()
topic_text_df.reset_index(level=0, inplace=True)
topic_text_df = topic_text_df.rename(columns={0: "topic_text", 'index': "topic"})

print(topic_text_df)
topic_results = nmf_model.transform(dtm)

df_messages['topic'] = topic_results.argmax(axis=1)
df_messages[['entryID', 'body', 'temp_body', 'topic']]


c:\users\bensg\pycharmprojects\network_groupdetection\venv\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning:

The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).



record size :  16308
    topic                                         topic_text
0       0  ['use auditory', '10x great', '10x', 'processi...
1       1  ['time', 'want', 'year', 'help', 'let', 'pm', ...
2       2  ['promotion unsubscribe', 'nrsnewsletternrscom...
3       3  ['supply', 'reference', 'price availability', ...
4       4  ['endeavor sincerely', 'bensgoldwebergmailcom ...
5       5  ['ifttt', 'advantage offer', 'sale', 'notifica...
6       6  ['salt', 'salt lake', 'city utah', 'lake city'...
7       7  ['bank', 'sign', 'banking', 'service', 'contac...
8       8  ['ca 90404', 'unsubscribe newsletter', '90404 ...
9       9  ['miss email', 'httptemcheggcomrescheggfa62f36...
10     10  ['management page', 'like unsubscribe', 'campa...
11     11  ['6777446', 'sunday', 'team', 'building 17', '...
12     12  ['sarl', 'rives clausen', 'rives', '2329', 'l2...
13     13  ['infopotomacpaddlesportscom', 'paddlesports',...
14     14  ['httpwwwcrackedcomnewsletterstaticbottom2gif'...


,entryID,body,temp_body,topic
0,00000000F0D1AD32B4212540B2740199443400840700C3...,I think it looks good. Can you reach out to N...,think look good reach nancy directly approve b...,1
1,00000000F0D1AD32B4212540B2740199443400840700C3...,\t\r\n\t\r\n\tThank you for electronically sig...,thank electronically sign movement earth trek ...,1
2,00000000F0D1AD32B4212540B2740199443400840700C3...,Congratulations! Your email address has been c...,congratulation email address confirm copy move...,1
3,00000000F0D1AD32B4212540B2740199443400840700C3...,‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌...,httpstwittercomiredirecturlhttps3a2f2ftwitterc...,12
4,00000000F0D1AD32B4212540B2740199443400840700C3...,"fyi, the core group of old guys are all out to...",fyi core group old guy today jon andy hopefull...,1
...,...,...,...,...
2469,00000000F0D1AD32B4212540B2740199443400840700C3...,\t <https://www.google.com/alerts?source=alert...,httpswwwgooglecomalertssourcealertsmailhlenglu...,1
2470,00000000F0D1AD32B4212540B2740199443400840700C3...,\t <https://www.google.com/alerts?source=alert...,httpswwwgooglecomalertssourcealertsmailhlenglu...,1
2471,00000000F0D1AD32B4212540B2740199443400840700C3...,\t <https://www.google.com/alerts?source=alert...,httpswwwgooglecomalertssourcealertsmailhlenglu...,1
2472,00000000F0D1AD32B4212540B2740199443400840700C3...,<http://www.causes.com/assets/email/color_hea...,httpwwwcausescomassetsemailcolorheaderwwhite13...,10


In [127]:
#THIS CHUNK GENERATES TOPIC COLORS AND MERGES IT WITH THE MASTER TABLEs
import random

topic_colors = {}
for x in range(topic_count):
    # topic_colors.update(x, hex_color)
    hex_color = "%06x" % random.randint(0, 0xFFFFFF)
    hex_color = "#" + hex_color
    topic_colors.update({ x: x , x : hex_color,})

topic_color_df = pd.DataFrame(topic_colors.items(), columns=['topic','color'])
topic_color_df = topic_color_df.merge(topic_text_df, left_on=['topic','topic'], right_on=['topic','topic'],  how='left')

print("TOPIC COLOR DF \n")
print(topic_color_df)

df_messages = df_messages.merge(topic_color_df, left_on=['topic','topic'], right_on=['topic','topic'],  how='left')
df_messages

TOPIC COLOR DF 

    topic    color                                         topic_text
0       0  #8d96a6  ['use auditory', '10x great', '10x', 'processi...
1       1  #294e96  ['time', 'want', 'year', 'help', 'let', 'pm', ...
2       2  #ac2af8  ['promotion unsubscribe', 'nrsnewsletternrscom...
3       3  #f6c0e4  ['supply', 'reference', 'price availability', ...
4       4  #21578b  ['endeavor sincerely', 'bensgoldwebergmailcom ...
5       5  #d29180  ['ifttt', 'advantage offer', 'sale', 'notifica...
6       6  #ce2381  ['salt', 'salt lake', 'city utah', 'lake city'...
7       7  #f7c0d3  ['bank', 'sign', 'banking', 'service', 'contac...
8       8  #603a0f  ['ca 90404', 'unsubscribe newsletter', '90404 ...
9       9  #eb9fe5  ['miss email', 'httptemcheggcomrescheggfa62f36...
10     10  #223492  ['management page', 'like unsubscribe', 'campa...
11     11  #9d6295  ['6777446', 'sunday', 'team', 'building 17', '...
12     12  #cd16d7  ['sarl', 'rives clausen', 'rives', '2329', 'l2...
13 

,entryID,conversationID,conversationIndex,createTime,recievedTime,ConversationTopic,subject,body,temp_body,topic,color,topic_text
0,00000000F0D1AD32B4212540B2740199443400840700C3...,0EB198BF1F35FB8AD6A5FF4CF4F39B3E,010195D751BE0EB198BF1F35FB8AD6A5FF4CF4F39B3E02...,2022-03-18 11:29:08.733000+00:00,2022-03-16 17:07:36+00:00,Lyme Disease Newsletter,Fwd: Lyme Disease Newsletter,I think it looks good. Can you reach out to N...,think look good reach nancy directly approve b...,1,#294e96,"['time', 'want', 'year', 'help', 'let', 'pm', ..."
1,00000000F0D1AD32B4212540B2740199443400840700C3...,5E18E9F8D63501DA96C678EBC29C4207,010152AF23855E18E9F8D63501DA96C678EBC29C4207,2022-03-18 11:29:08.747000+00:00,2022-03-16 18:19:38+00:00,Please Confirm your E-Signature on the MOVEMEN...,Please Confirm your E-Signature on the MOVEMEN...,\t\r\n\t\r\n\tThank you for electronically sig...,thank electronically sign movement earth trek ...,1,#294e96,"['time', 'want', 'year', 'help', 'let', 'pm', ..."
2,00000000F0D1AD32B4212540B2740199443400840700C3...,421B2A78311B89A8CDD0B3E5B0990847,0101F87FC610421B2A78311B89A8CDD0B3E5B0990847,2022-03-18 11:29:08.747000+00:00,2022-03-16 18:19:59+00:00,Ben Goldweber's MOVEMENT / EARTH TREKS / PLANE...,Ben Goldweber's MOVEMENT / EARTH TREKS / PLANE...,Congratulations! Your email address has been c...,congratulation email address confirm copy move...,1,#294e96,"['time', 'want', 'year', 'help', 'let', 'pm', ..."
3,00000000F0D1AD32B4212540B2740199443400840700C3...,2A5E9D225B351EF75CD519203EBFA981,0100EB198CF12A5E9D225B351EF75CD519203EBFA981,2022-03-18 11:29:08.747000+00:00,2022-03-17 03:58:14+00:00,Rachel Vindman 🌻 Tweeted: This hateful bitch w...,Rachel Vindman 🌻 Tweeted: This hateful bitch w...,‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌...,httpstwittercomiredirecturlhttps3a2f2ftwitterc...,12,#cd16d7,"['sarl', 'rives clausen', 'rives', '2329', 'l2..."
4,00000000F0D1AD32B4212540B2740199443400840700C3...,069A757C1C0E6498ADA205F5A8C01664,0101C44EF4DF069A757C1C0E6498ADA205F5A8C0166402...,2022-03-18 11:29:08.762000+00:00,2022-03-17 11:17:09+00:00,Old Guy Attaining Workouts,Re: Old Guy Attaining Workouts,"fyi, the core group of old guys are all out to...",fyi core group old guy today jon andy hopefull...,1,#294e96,"['time', 'want', 'year', 'help', 'let', 'pm', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2469,00000000F0D1AD32B4212540B2740199443400840700C3...,7F3069F23535B0BC882BB5EB8AF443F7,010312E5BF397F3069F23535B0BC882BB5EB8AF443F7,2022-03-18 11:30:04.708000+00:00,2015-06-07 00:01:11+00:00,Google Alert - Montgomery County,Google Alert - Montgomery County,\t <https://www.google.com/alerts?source=alert...,httpswwwgooglecomalertssourcealertsmailhlenglu...,1,#294e96,"['time', 'want', 'year', 'help', 'let', 'pm', ..."
2470,00000000F0D1AD32B4212540B2740199443400840700C3...,76F731506D82C33902447C2751D0A181,0101E34EB32576F731506D82C33902447C2751D0A181,2022-03-18 11:30:04.713000+00:00,2015-06-07 00:01:29+00:00,Google Alert - Water,Google Alert - Water,\t <https://www.google.com/alerts?source=alert...,httpswwwgooglecomalertssourcealertsmailhlenglu...,1,#294e96,"['time', 'want', 'year', 'help', 'let', 'pm', ..."
2471,00000000F0D1AD32B4212540B2740199443400840700C3...,9F928AC27CF204FE33DB0704E3BC66B3,01015BFF9FCA9F928AC27CF204FE33DB0704E3BC66B3,2022-03-18 11:30:04.718000+00:00,2015-06-07 00:46:14+00:00,Google Alert - Landscaping,Google Alert - Landscaping,\t <https://www.google.com/alerts?source=alert...,httpswwwgooglecomalertssourcealertsmailhlenglu...,1,#294e96,"['time', 'want', 'year', 'help', 'let', 'pm', ..."
2472,00000000F0D1AD32B4212540B2740199443400840700C3...,CE78DE94198B7A95F08B2A86E4F22FA6,0102756876F3CE78DE94198B7A95F08B2A86E4F22FA6,2022-03-18 11:30:04.726000+00:00,2015-06-07 09:58:20+00:00,Welcome to Causes! Get involved right away.,Welcome to Causes! Get involved right away.,<http://www.causes.com/assets/email/color_hea...,httpwwwcausescomassetsemailcolorheaderwwhite13...,10,#223492,"['management page', 'like 

In [128]:
print("\nShape")
print("Recipient Shape is " + str(df_recipients.shape))
print("Message Shape is " +  str(df_messages.shape))


Shape
Recipient Shape is (21029, 3)
Message Shape is (2474, 12)


In [129]:
df_recipients = df_messages.merge(df_recipients, left_on=['entryID','entryID'], right_on=['entryID','entryID'],  how='left')
df_recipients = df_recipients.reset_index(drop=True)
df_recipients

,entryID,conversationID,conversationIndex,createTime,recievedTime,ConversationTopic,subject,body,temp_body,topic,color,topic_text,sender,recipient
0,00000000F0D1AD32B4212540B2740199443400840700C3...,0EB198BF1F35FB8AD6A5FF4CF4F39B3E,010195D751BE0EB198BF1F35FB8AD6A5FF4CF4F39B3E02...,2022-03-18 11:29:08.733000+00:00,2022-03-16 17:07:36+00:00,Lyme Disease Newsletter,Fwd: Lyme Disease Newsletter,I think it looks good. Can you reach out to N...,think look good reach nancy directly approve b...,1,#294e96,"['time', 'want', 'year', 'help', 'let', 'pm', ...",Barb Siegel,Ben Goldweber
1,00000000F0D1AD32B4212540B2740199443400840700C3...,5E18E9F8D63501DA96C678EBC29C4207,010152AF23855E18E9F8D63501DA96C678EBC29C4207,2022-03-18 11:29:08.747000+00:00,2022-03-16 18:19:38+00:00,Please Confirm your E-Signature on the MOVEMEN...,Please Confirm your E-Signature on the MOVEMEN...,\t\r\n\t\r\n\tThank you for electronically sig...,thank electronically sign movement earth trek ...,1,#294e96,"['time', 'want', 'year', 'help', 'let', 'pm', ...",Movement / Planet Granite / Earth Treks,bensgoldweber@gmail.com
2,00000000F0D1AD32B4212540B2740199443400840700C3...,421B2A78311B89A8CDD0B3E5B0990847,0101F87FC610421B2A78311B89A8CDD0B3E5B0990847,2022-03-18 11:29:08.747000+00:00,2022-03-16 18:19:59+00:00,Ben Goldweber's MOVEMENT / EARTH TREKS / PLANE...,Ben Goldweber's MOVEMENT / EARTH TREKS / PLANE...,Congratulations! Your email address has been c...,congratulation email address confirm copy move...,1,#294e96,"['time', 'want', 'year', 'help', 'let', 'pm', ...",Movement / Planet Granite / Earth Treks,bensgoldweber@gmail.com
3,00000000F0D1AD32B4212540B2740199443400840700C3...,2A5E9D225B351EF75CD519203EBFA981,0100EB198CF12A5E9D225B351EF75CD519203EBFA981,2022-03-18 11:29:08.747000+00:00,2022-03-17 03:58:14+00:00,Rachel Vindman 🌻 Tweeted: This hateful bitch w...,Rachel Vindman 🌻 Tweeted: This hateful bitch w...,‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌...,httpstwittercomiredirecturlhttps3a2f2ftwitterc...,12,#cd16d7,"['sarl', 'rives clausen', 'rives', '2329', 'l2...",Twitter,Ben Goldweber
4,00000000F0D1AD32B4212540B2740199443400840700C3...,069A757C1C0E6498ADA205F5A8C01664,0101C44EF4DF069A757C1C0E6498ADA205F5A8C0166402...,2022-03-18 11:29:08.762000+00:00,2022-03-17 11:17:09+00:00,Old Guy Attaining Workouts,Re: Old Guy Attaining Workouts,"fyi, the core group of old guys are all out to...",fyi core group old guy today jon andy hopefull...,1,#294e96,"['time', 'want', 'year', 'help', 'let', 'pm', ...",Dave Donoho,Ben Goldweber
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21026,00000000F0D1AD32B4212540B2740199443400840700C3...,7F3069F23535B0BC882BB5EB8AF443F7,010312E5BF397F3069F23535B0BC882BB5EB8AF443F7,2022-03-18 11:30:04.708000+00:00,2015-06-07 00:01:11+00:00,Google Alert - Montgomery County,Google Alert - Montgomery County,\t <https://www.google.com/alerts?source=alert...,httpswwwgooglecomalertssourcealertsmailhlenglu...,1,#294e96,"['time', 'want', 'year', 'help', 'let', 'pm', ...",Google Alerts,BensGoldweber@gmail.com
21027,00000000F0D1AD32B4212540B2740199443400840700C3...,76F731506D82C33902447C2751D0A181,0101E34EB32576F731506D82C33902447C2751D0A181,2022-03-18 11:30:04.713000+00:00,2015-06-07 00:01:29+00:00,Google Alert - Water,Google Alert - Water,\t <https://www.google.com/alerts?source=alert...,httpswwwgooglecomalertssourcealertsmailhlenglu...,1,#294e96,"['time', 'want', 'year', 'help', 'let', 'pm', ...",Google Alerts,BensGoldweber@gmail.com
21028,00000000F0D1AD32B4212540B2740199443400840700C3...,9F928AC27CF204FE33DB0704E3BC66B3,01015BFF9FCA9F928AC27CF204FE33DB0704E3BC66B3,2022-03-18 11:30:04.718000+00:00,2015-06-07 00:46:14+00:00,Google Alert - Landscaping,Google Alert - Landscaping,\t <https://www.google.com/alerts?source=alert...,httpswwwgooglecomalertssourcealertsmailhlenglu...,1,#294e96,"['time', 'want', 'year', 'help', 'let', 'pm', ...",Google Alerts,BensGoldweber@gmail.com
21029,00000000F0D1AD32B4212540B2740199443400840700C3...,CE78DE94198B7A95F

In [130]:
df_recipients = df_recipients.groupby(['sender',  'recipient', 'topic', 'color']).size()
df_recipients = df_recipients.to_frame(name = 'size').reset_index()
df_recipients = df_recipients.loc[df_recipients['size'] != 1]

nan_value = float("NaN")
df_recipients.replace("", nan_value, inplace=True)
df_recipients = df_recipients.dropna()
df_recipients

,sender,recipient,topic,color,size
0,3point5 Customer Support,Ben Goldweber,1,#294e96,3
2,3point5 Customer Support,bensgoldweber@gmail.com,1,#294e96,2
10,Adele Barbieri,Alex Nielson,11,#9d6295,2
12,Adele Barbieri,Ben Goldweber,11,#9d6295,3
16,Adele Barbieri,Joe Mornini,11,#9d6295,3
...,...,...,...,...,...
9996,matthew markoff,lucyww@uga.edu,1,#294e96,2
9997,matthew markoff,omurphy@visi.org,1,#294e96,2
10002,noreply@usairways.com,BENSGOLDWEBER@GMAIL.COM,8,#603a0f,3
10005,ssgztaylor@yahoo.com,Ben Goldweber,13,#839827,2


In [131]:
#THIS CHUNK SAVES TO SQLITE THE DATA PULLED FROM OUTLOOK

sql_con = sqlite3.connect("test_db_sqlite.db")

# Write the new DataFrame to a new SQLite table
df_recipients.to_sql("compressed_data", sql_con, if_exists="replace")
# topic_color_df = pd.read_sql_query(topic_color_df, sql_con)
topic_color_df.to_sql("topic_color_df", sql_con, if_exists="replace")

# df_recipients = pd.DataFrame(pd_recipients)
sql_con.close()

In [132]:
#Start from here to cut down on processing time

sql_con = sqlite3.connect("test_db_sqlite.db")


recipient_query = 'select * from compressed_data;'
topic_query = 'select * from topic_color_df;'

df_recipients = pd.read_sql_query(recipient_query, sql_con)
topic_color_df = pd.read_sql_query(topic_query, sql_con)

sql_con.close()

df_recipients = df_recipients.drop('index', 1)
topic_color_df = topic_color_df.drop('index', 1)


C:\Users\bensg\AppData\Local\Temp\ipykernel_19152\2604522456.py:14: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

C:\Users\bensg\AppData\Local\Temp\ipykernel_19152\2604522456.py:15: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



In [133]:
print("\n MESSAGES DF \n")
#print(df_messages.head())
print("\n RECIPIENTS DF \n")
print(df_recipients.head())

df_counts = df_recipients.groupby(['sender',  'recipient', 'topic','color']).size().reset_index().rename(columns={0: 'count'})

df_counts


 MESSAGES DF 


 RECIPIENTS DF 

                     sender                recipient  topic    color  size
0  3point5 Customer Support            Ben Goldweber      1  #294e96     3
1  3point5 Customer Support  bensgoldweber@gmail.com      1  #294e96     2
2            Adele Barbieri             Alex Nielson     11  #9d6295     2
3            Adele Barbieri            Ben Goldweber     11  #9d6295     3
4            Adele Barbieri              Joe Mornini     11  #9d6295     3


,sender,recipient,topic,color,count
0,3point5 Customer Support,Ben Goldweber,1,#294e96,1
1,3point5 Customer Support,bensgoldweber@gmail.com,1,#294e96,1
2,Adele Barbieri,Alex Nielson,11,#9d6295,1
3,Adele Barbieri,Ben Goldweber,11,#9d6295,1
4,Adele Barbieri,Joe Mornini,11,#9d6295,1
...,...,...,...,...,...
2617,matthew markoff,lucyww@uga.edu,1,#294e96,1
2618,matthew markoff,omurphy@visi.org,1,#294e96,1
2619,noreply@usairways.com,BENSGOLDWEBER@GMAIL.COM,8,#603a0f,1
2620,ssgztaylor@yahoo.com,Ben Goldweber,13,#839827,1


In [134]:
#node df generator
df_node_temp =  df_recipients

#build node list off of to/from table. Adding a new feature called count driven by node edge counts
df1 = df_node_temp['sender']
df2 = df_node_temp['recipient']
df_nodes = df1.append(df2)
df_nodes.reset_index()
#df_nodes.set_axis(['name'], axis=1)
df_nodes = df_nodes.to_frame()
print("Dataframe columns:", df_nodes.columns)

#df_nodes['individuals'] = df_nodes.columns[0]
df_nodes = df_nodes.rename(columns={0: "individuals"})

#print("Dataframe columns:", df_nodes.columns)
df_nodes = df_nodes['individuals'].value_counts()

df_nodes = df_nodes.to_frame()
df_nodes = df_nodes.rename(columns={'individuals': "count"})
df_nodes.index.name = 'individual'
df_nodes.reset_index(inplace=True)
print(df_nodes)


Dataframe columns: RangeIndex(start=0, stop=1, step=1)
                              individual  count
0                            Lucy Martin    520
1                          Nicole Norton    475
2                            Matt Gresko    297
3        Lucy Martin (via Google Sheets)    236
4                             Lisa Thuro    223
..                                   ...    ...
885                      Obsidian Portal      1
886                              Tanya K      1
887  Optimum Nutrition and ProMotive.com      1
888                              Pandora      1
889                  bsg1232@verizon.net      1

[890 rows x 2 columns]


C:\Users\bensg\AppData\Local\Temp\ipykernel_19152\3212497709.py:7: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [135]:
df_recipients

,sender,recipient,topic,color,size
0,3point5 Customer Support,Ben Goldweber,1,#294e96,3
1,3point5 Customer Support,bensgoldweber@gmail.com,1,#294e96,2
2,Adele Barbieri,Alex Nielson,11,#9d6295,2
3,Adele Barbieri,Ben Goldweber,11,#9d6295,3
4,Adele Barbieri,Joe Mornini,11,#9d6295,3
...,...,...,...,...,...
2617,matthew markoff,lucyww@uga.edu,1,#294e96,2
2618,matthew markoff,omurphy@visi.org,1,#294e96,2
2619,noreply@usairways.com,BENSGOLDWEBER@GMAIL.COM,8,#603a0f,3
2620,ssgztaylor@yahoo.com,Ben Goldweber,13,#839827,2


In [136]:

edges_table = [df_recipients["sender"], df_recipients["recipient"], df_recipients['topic'], df_recipients['color'], df_recipients['size']]
# df_recipients

headers = ["source", "target", "topic","color", "size"]
df_edges = pd.concat(edges_table, axis=1, keys=headers)
print(df_edges)


                        source                   target  topic    color  size
0     3point5 Customer Support            Ben Goldweber      1  #294e96     3
1     3point5 Customer Support  bensgoldweber@gmail.com      1  #294e96     2
2               Adele Barbieri             Alex Nielson     11  #9d6295     2
3               Adele Barbieri            Ben Goldweber     11  #9d6295     3
4               Adele Barbieri              Joe Mornini     11  #9d6295     3
...                        ...                      ...    ...      ...   ...
2617           matthew markoff           lucyww@uga.edu      1  #294e96     2
2618           matthew markoff         omurphy@visi.org      1  #294e96     2
2619     noreply@usairways.com  BENSGOLDWEBER@GMAIL.COM      8  #603a0f     3
2620      ssgztaylor@yahoo.com            Ben Goldweber     13  #839827     2
2621            zachery taylor            Jonathon Hall     13  #839827     2

[2622 rows x 5 columns]


In [137]:

count_mean_nodes = df_nodes['count'].mean()
count_std_dev_nodes = df_nodes['count'].std()
df_nodes['edge_count_z-score'] = (df_nodes['count'] - count_mean_nodes) / count_std_dev_nodes
values_plus_minus_one_z_score = len(df_nodes[df_nodes['edge_count_z-score'].between(-1, 1) == True])
percent_values_plus_minus_one_z_score = values_plus_minus_one_z_score / len(df_nodes) * 100
z_score_distribution_std_dev = round(df_nodes['edge_count_z-score'].std(), 2)
ser = Series(df_nodes['count'])
bins = np.quantile(np.unique(ser), np.linspace(0, 1, 11))

num_rows = np.shape(bins)[0]
bin_count = num_rows
starting_count = 5
size_labels = range(starting_count, (bin_count + starting_count) - 1, 1)

df_nodes['bins'] = pd.cut(ser, bins, include_lowest=True, labels=size_labels)
# df_nodes

#Log smooth the bins
# df_nodes['bins'] = df_nodes['bins'].astype(np.int8)
df_nodes['log_count'] = np.log(df_nodes['count'])
#
print(df_nodes)

                              individual  count  edge_count_z-score bins  \
0                            Lucy Martin    520           17.503965   14   
1                          Nicole Norton    475           15.971838   14   
2                            Matt Gresko    297            9.911425   14   
3        Lucy Martin (via Google Sheets)    236            7.834543   13   
4                             Lisa Thuro    223            7.391928   13   
..                                   ...    ...                 ...  ...   
885                      Obsidian Portal      1           -0.166564    5   
886                              Tanya K      1           -0.166564    5   
887  Optimum Nutrition and ProMotive.com      1           -0.166564    5   
888                              Pandora      1           -0.166564    5   
889                  bsg1232@verizon.net      1           -0.166564    5   

     log_count  
0     6.253829  
1     6.163315  
2     5.693732  
3     5.463832  
4 

In [138]:
df_nodes

,individual,count,edge_count_z-score,bins,log_count
0,Lucy Martin,520,17.503965,14,6.253829
1,Nicole Norton,475,15.971838,14,6.163315
2,Matt Gresko,297,9.911425,14,5.693732
3,Lucy Martin (via Google Sheets),236,7.834543,13,5.463832
4,Lisa Thuro,223,7.391928,13,5.407172
...,...,...,...,...,...
885,Obsidian Portal,1,-0.166564,5,0.000000
886,Tanya K,1,-0.166564,5,0.000000
887,Optimum Nutrition and ProMotive.com,1,-0.166564,5,0.000000
888,Pandora,1,-0.166564,5,0.000000


In [139]:

df_edges

,source,target,topic,color,size
0,3point5 Customer Support,Ben Goldweber,1,#294e96,3
1,3point5 Customer Support,bensgoldweber@gmail.com,1,#294e96,2
2,Adele Barbieri,Alex Nielson,11,#9d6295,2
3,Adele Barbieri,Ben Goldweber,11,#9d6295,3
4,Adele Barbieri,Joe Mornini,11,#9d6295,3
...,...,...,...,...,...
2617,matthew markoff,lucyww@uga.edu,1,#294e96,2
2618,matthew markoff,omurphy@visi.org,1,#294e96,2
2619,noreply@usairways.com,BENSGOLDWEBER@GMAIL.COM,8,#603a0f,3
2620,ssgztaylor@yahoo.com,Ben Goldweber,13,#839827,2


In [140]:
#THIS CHUNK SAVES TO SQLITE THE DATA PULLED FROM OUTLOOK

sql_con = sqlite3.connect("test_db_sqlite.db")

# Write the new DataFrame to a new SQLite table
df_nodes.to_sql("df_nodes", sql_con, if_exists="replace")
# topic_color_df = pd.read_sql_query(topic_color_df, sql_con)
df_edges.to_sql("df_edges", sql_con, if_exists="replace")

# df_recipients = pd.DataFrame(pd_recipients)
sql_con.close()

In [141]:
sql_con = sqlite3.connect("test_db_sqlite.db")

edges_q = 'select * from df_edges;'
nodes_q = 'select * from df_nodes;'
topics_q = 'select * from topic_color_df;'

df_edges = pd.read_sql_query(edges_q, sql_con)
df_nodes = pd.read_sql_query(nodes_q, sql_con)

topic_color_df = pd.read_sql_query(topics_q, sql_con)

sql_con.close()

df_edges = df_edges.drop('index', 1)
df_nodes = df_nodes.drop('index', 1)
topic_color_df = topic_color_df.drop('index', 1)


C:\Users\bensg\AppData\Local\Temp\ipykernel_19152\3381265128.py:14: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

C:\Users\bensg\AppData\Local\Temp\ipykernel_19152\3381265128.py:15: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

C:\Users\bensg\AppData\Local\Temp\ipykernel_19152\3381265128.py:16: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



GRAPH GENERATION

In [142]:
import pandas as pd

import dash
import json

from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from dash import dash_table


import dash_cytoscape as cyto
cyto.load_extra_layouts()

app = dash.Dash(__name__)
server = app.server

edges = df_edges
cy_edges = []

# nodes = set()
# nodes = df_nodes

nodes = set()
cy_nodes = []

# headers = ["source", "target", "weight", "topic"]


#build nodes
for index, row in df_nodes.iterrows():
    individual, bins = row['individual'], row['log_count']
    nodes.add(individual)
    cy_nodes.append({"data": {"id": individual, "label": individual, 'weight': bins*10, }})


#build edges
for index, row in edges.iterrows():
    source, target, topic, weight, topic_color = row['source'], row['target'], row['topic'], row['size'], row['color']
    cy_edges.append({
        'data': {
            'source': source,
            'target': target,
            'topic': topic,
            'weight': weight,
            'topic_color': topic_color
        }
    })


def generate_table(dataframe, max_rows=200):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                 html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))

        ])
    ])


# define stylesheet
n_stylesheet = [
    {
        "selector": 'node',  #For all nodes
        'style': {
            "opacity": 0.9,
            "height": "data(weight)",
            'width': 'data(weight)',
            "label": "data(label)",  #Label of node to display
            "background-color": "#07ABA0",  #node color
            "color": "#008B80"  #node label color
        }
    },
    {
        "selector": 'edge',  #For all edges
        "style": {
            "target-arrow-color": "#C5D3E2",  #Arrow color
            "target-arrow-shape": "triangle",  #Arrow shape
            "line-color": "data(topic_color)",  #edge color
            'arrow-scale': 2,  #Arrow size
            'curve-style': 'bezier'  #Default curve-If it is style, the arrow will not be displayed, so specify it
        }
    },
]

page_styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll',
        'min-height': '50px'
    }
}

edge_legend = topic_color_df[['topic','color']]

app.layout = html.Div(children=[
    html.H4(children='Communication patters'),

    html.Div(
        children=[
            html.Div(children=[
            cyto.Cytoscape(
                id='cytoscape',
                elements=cy_edges + cy_nodes,
                style={
                    'height': '95vh',
                    'width': '100%'
                },
                 layout={'name': 'grid'},
                stylesheet=n_stylesheet
            )], style={'width': '50%'}),
            html.Div(children=[
                    dcc.Dropdown(id='dropdown_topic',
                 options=[{'label': topic.capitalize(), 'value': topic}
                         for topic in ['0', '1', '2', '3', '4']
                         ]
                 ),

    dcc.Dropdown(
        id='dropdown-update-layout',
        options=[
            {'label': 'random',
             'value': 'random'},
            {'label': 'grid',
             'value': 'grid'},
            {'label': 'circle',
             'value': 'circle'},
            {'label': 'concentric',
             'value': 'concentric'},
            {'label': 'breadthfirst - Hiearchy',
             'value': 'breadthfirst'},
            {'label': 'klay - Force Directed',
             'value': 'klay'},
            {'label': 'cose - Force Directed',
             'value': 'cose'},
            {'label': 'cose-bilkent - Force Directed',
             'value': 'cose-bilkent'},
            {'label': 'cola - Force Directed',
             'value': 'cola'},
            {'label': 'spread - Force Directed',
             'value': 'spread'},
            {'label': 'dagre - Hiearchy',
             'value': 'dagre'}
        ], value='circle'
    ),
                html.H4(children='NodeData'),
                html.P(id='cytoscape_element_info_output'),
                html.P(id='cytoscape-tapEdgeData-output'),
                html.H4(children='Legend'),
                dash_table.DataTable(
                    data=edge_legend.to_dict('records'),
                    columns=[{"name": i, "id": i} for i in edge_legend.columns],
                    style_cell={'textAlign': 'left'},
                    style_data_conditional=[
                        {'if': {'row_index': i, 'column_id': 'color'},
                         'background-color': edge_legend['color'][i],
                         'color': edge_legend['color'][i]} for i in range(edge_legend.shape[0])
                    ]
                     )
        ], style={'width': '50%'})
        ], style={'display': 'flex', 'flex-direction': 'row'}),

                dash_table.DataTable(
                    data=topic_color_df.to_dict('records'),
                    columns=[{"name": i, "id": i} for i in topic_color_df.columns],
                    style_cell={'textAlign': 'left'},
                    style_data_conditional=[
                        {'if': {'row_index': i, 'column_id': 'color'},
                         'background-color': topic_color_df['color'][i],
                         'color': topic_color_df['color'][i]} for i in range(topic_color_df.shape[0])
                    ]
                     )
])


In [143]:

# @app.callback(Output('cytoscape-mouseoverNodeData-output', 'children'),
#               Input('cytoscape', 'mouseoverNodeData'))
# def displayTapNodeData(data):
#     if data:
#         return "You recently hovered over the city: " + data['label']


@app.callback(Output('cytoscape', 'layout'),
              Input('dropdown-update-layout', 'value'))
def update_layout(layout):
        return {
            'name': layout,
        }


@app.callback(Output('cytoscape_element_info_output', 'children'),
              Input('cytoscape', 'tapNodeData'))
def displayTapNodeData(data):
    if data:
        return "You recently clicked/tapped node: " + data['label']


@app.callback(Output('cytoscape-tapEdgeData-output', 'children'),
              Input('cytoscape', 'tapEdgeData'))
def displayTapEdgeData(data):
    if data:
        return json.dumps(data, indent=2)
        # response =  "You recently clicked/tapped the email of topic " + str(data['topic']) +  " between " + data['source'].upper() + " and " + data['target'].upper()  + " Which has a subject of- " + data['subject'] + "\n" + "Body: " + data['body']
        # return response

            #"You recently clicked/tapped the email between " + data['source'].upper() + " and " + data['target'].upper()  + " Which has a subject of " + data['subject']

        #"You recently clicked/tapped the email between " + data['source'].upper() + " and " + data['target'].upper()  + " Which has a subject of " +  data['subject']



if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Mar/2022 12:35:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 12:35:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 12:35:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 12:35:51] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 12:35:51] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 12:35:51] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 12:35:53] "POST /_dash-update-component HTTP/1.1" 200 -
12